In [1]:
import pandas as pd
import numpy as np
from gensim import corpora
from nltk.tokenize import RegexpTokenizer
# from nltk.corpus import stopwords

In [2]:
# read in file
df = pd.read_csv('./data/combined.csv')

In [3]:
# I want to be able to see the entire title of a notice 
# set display options to full column widths
pd.set_option('display.max_colwidth', None)

In [4]:
df.head()

,noticeId,title,solicitationNumber,department,subTier,office,postedDate,type,baseType,archiveType,...,award,pointOfContact,description,organizationType,officeAddress,placeOfPerformance,additionalInfoLink,uiLink,links,resourceLinks
0,ffe3c07bab8b4a7d9be583fd4dd96c4a,REQUEST FOR INFORMATION - NEW DESIGN BOOKLET DIE CUTTING INSERT ASSEMBLY,RFI-042720,UNITED STATES GOVERNMENT PUBLISHING OFFICE,UNITED STATES GOVERNMENT PUBLISHING OFFICE,Acquisition Services,2020-04-27,Sources Sought,Sources Sought,auto15,...,NaN,"[{'fax': '', 'type': 'primary', 'email': 'rgraham@gpo.gov', 'phone': '2025120032', 'title': None, 'fullName': 'Rosalyn Graham'}, {'fax': '', 'type': 'secondary', 'email': 'rortega@gpo.gov', 'phone': '2025120803', 'title': None, 'fullName': 'Ronald Ortega'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=ffe3c07bab8b4a7d9be583fd4dd96c4a,OFFICE,"{'zipcode': '20401', 'city': 'Washington', 'countryCode': 'USA', 'state': 'DC'}","{'state': {'code': 'DC', 'name': 'District of Columbia'}, 'zip': '20401', 'country': {'code': 'USA', 'name': 'UNITED STATES'}}",NaN,https://beta.sam.gov/opp/ffe3c07bab8b4a7d9be583fd4dd96c4a/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=ffe3c07bab8b4a7d9be583fd4dd96c4a&limit=1', 'hreflang': None, 'media': None, 'title': None, 'type': None, 'deprecation': None}]","['https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/bd2b3b5e86474278ae1ea1a163ccbfd7/download?api_key=null&token=', 'https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/774a16e89e4d431e85b0eba1a510a24c/download?api_key=null&token=', 'https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/b62d72d82537413f8537bb19160d49bd/download?api_key=null&token=']"
1,ff5e29951ed14b0386ea3291fa6c573f,Audiovisual Suite for Large Auditorium - Maxwell AFB,FA330020X0001,DEPT OF DEFENSE,DEPT OF THE AIR FORCE,FA3300 42 CONS CC,2020-04-27,Sources Sought,Sources Sought,auto15,...,NaN,"[{'fax': '3349532198', 'type': 'primary', 'email': 'brandon.osborne.6@us.af.mil', 'phone': '3349537835', 'title': None, 'fullName': 'Brandon Osborne'}, {'fax': '3349532198', 'type': 'secondary', 'email': 'jeremy.kersey.1@us.af.mil', 'phone': '3349533526', 'title': None, 'fullName': 'Jeremy B. Kersey'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=ff5e29951ed14b0386ea3291fa6c573f,OFFICE,"{'zipcode': '36112-6334', 'city': 'MAXWELL AFB', 'countryCode': 'USA', 'state': 'AL'}","{'city': {'code': '51000', 'name': 'Montgomery'}, 'state': {'code': 'AL', 'name': 'Alabama'}, 'zip': '36112', 'country': {'code': 'USA', 'name': 'UNITED STATES'}}",NaN,https://beta.sam.gov/opp/ff5e29951ed14b0386ea3291fa6c573f/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=ff5e29951ed14b0386ea3291fa6c573f&limit=1', 'hreflang': None, 'media': None, 'title': None, 'type': None, 'deprecation': None}]",['https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/c2a6338838ed4ebfa7d70e91221ffc21/download?api_key=null&token=']
2,ff2a16aad70246d1ba10e639c11456d4,Metrology Equipment Move,SP4703-20-Q-0045,DEPT OF DEFENSE,DEFENSE LOGISTICS AGENCY (DLA),DCSO RICHMOND,2020-04-27,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,auto15,...,NaN,"[{'fax': '', 'type': 'primary', 'email': 'Kiara.leslie@dla.mil', 'phone': '8042796566', 'title': None, 'fullName': 'Kiara Leslie'}, {'fax': '', 'type': 'secondary', 'email': 'mary.washington@dla.mil', 'phone': '8042795628', 'title': None, 'fullName': 'Mary A. Washington, Contracting Officer'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=ff2a16aad70246d1ba10e639c11456d4,OFFICE,"{'zipcode': '23297-5441', 'city': 'RICHMOND', 'countryCode': 'USA', 'state': 'VA'}","{'city': {'code': '67000', 'name': 'Richmond'}, 'state': {'code': 'VA', 'name': 'Virginia'}, 'zip': '23219', 'country': {'code': 'USA', 'name': 'UNITED STATES'}}",NaN,https://beta.sam.gov/opp/ff2a16aad70246d1ba10e639c11456d4/view,"[

#### Create the Corpus

The recommender is built on the text contained in the 'title' column of the dataframe.

**Inspect the type and base type of all notifications to decide if all types should be included in the dictionary.**

In [22]:
# list types of notices in the contract opportunities dataframe
df['type'].value_counts()

Combined Synopsis/Solicitation    18831
Solicitation                       7705
Presolicitation                    7043
Award Notice                       6106
Sources Sought                     4091
Special Notice                     2843
Justification                      1077
Sale of Surplus Property             43
Intent to Bundle Requirements         2
Name: type, dtype: int64

In [23]:
# list base types of notices in the contract opportunities dataframe
df['baseType'].value_counts()

Combined Synopsis/Solicitation      20057
Presolicitation                      7828
Solicitation                         7336
Sources Sought                       4803
Award Notice                         3978
Special Notice                       2836
Justification                         855
Sale of Surplus Property               43
Justification and Approval (J&A)        5
Name: baseType, dtype: int64


Since the title of all types of contract opportunities may contain pertinent information to find similarities and dissimilarities between different documents, the complete dataset will serve as the basis for the dictionary.

---

#### Isolate Titles for NLP

In [24]:
titles = df['title']

Because of the specific, succinct nature of the titles of notifications we are working with, to enrich context **the department columns were combined with the text of the title.** 

In [25]:
# fill NaN with empty string
df['department'] = df['department'].fillna(value='')
df['subTier'] = df['subTier'].fillna(value='')

In [26]:
# add department and subTier column to title column
df['combined'] = df['title'] + ' ' + df['department'] + ' ' + df['subTier']

In [27]:
df['combined'].isna().sum()

0

In [28]:
# isolate corpus of interest
combined = df['combined']
combined.shape

(47741,)

In [29]:
combined.sample(25)

24268                                                                                                                                                                                                 53--O-RING DEPT OF DEFENSE DEFENSE LOGISTICS AGENCY (DLA)
18350                                                                                                                                                                                        Computer Test Data Component DEPT OF DEFENSE DEPT OF THE AIR FORCE
2315                                                                                                                                                             6515--COVID-19 PPE N95 SDC     VETERANS AFFAIRS, DEPARTMENT OF VETERANS AFFAIRS, DEPARTMENT OF
8421                                                                                                                                                                                       42--EXTINGUISHER,FIRE DEPT OF DEFENSE DEFENSE

In [12]:
combined.tail()

43449                                                                            PAD, RUBBER DEPT OF DEFENSE DEPT OF THE NAVY
43450                                        Cabling Services VETERANS AFFAIRS, DEPARTMENT OF VETERANS AFFAIRS, DEPARTMENT OF
43451    J041--Ice Machines and Retherm Unit Service Contract VETERANS AFFAIRS, DEPARTMENT OF VETERANS AFFAIRS, DEPARTMENT OF
43452                                                  70--OPTICAL READER,DATA DEPT OF DEFENSE DEFENSE LOGISTICS AGENCY (DLA)
43453                 Joint Light Tactical Vehicle (JLTV) Family of Vehicles (FoV) Follow-On DEPT OF DEFENSE DEPT OF THE ARMY
Name: combined, dtype: object

#### Clean and tokenize text


I tried tagging entities with spaCy for the sake of easily removing dates, numbers and addresses. However, since entity tagging looks for capitalizations and this dataset uses capital letters inconsistently, it did not work well. 

In [3]:
# define function to tokenize a column

def tokenizer_function(column):
    """
    Takes in a text column
        tokenizes the text in each row
        using pattern [[a-zA-Z]\w+]
        which matches every lowercase and upperase character between a-z that are word characters
    Returns list of strings
    """
    
    # instantiate empty list of tokenized text
    texts = []
    
    # define tokenizer pattern
    pattern = '[a-zA-Z]\w+'
    # instantiate tokenizer
    tokenizer = RegexpTokenizer(pattern=pattern)
    
    # create for loop to tokenize each row and add the list of tokens to texts
    for text in column:
        tokens = tokenizer.tokenize(text)
        
        # transform tokens into lower case strings
        tokens = [token.lower() for token in tokens]
        texts.append(tokens)
    return texts

In [31]:
# tokenize combined documents
tokenized_combined = tokenizer_function(combined)

In [32]:
# save tokenized combined documents for future use
tokenized_combined.to_csv('./data/tokenized_combined.csv', index=False)

In [10]:
# tokenize titles
tokenized_titles = tokenizer_function(df['title'])

In [8]:
# # save tokenized titles
# tokenized_titles.to_csv('./data/tokenized_titles.csv', index=False)

In [9]:
# data = pd.read_csv('./data/tokenized_titles.csv')
# data.head()

/Users/bibor/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (37,38,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,request,for,information,new,design,booklet,die,cutting,insert,assembly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,audiovisual,suite,for,large,auditorium,maxwell,afb,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,metrology,equipment,move,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,inner,inflatable,assy,lpu,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cradle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# write tokenized_titles into a file

# with open('tokenized_titles.txt', 'w') as f:
#     for item in tokenized_titles:
#         f.write(str(item))
        
# source: https://stackoverflow.com/questions/33686747/save-a-list-to-a-txt-file

In [ ]:
# remove stop words

In [22]:
# subset of text documents to try out different nlp methods
titles_100 = titles.iloc[:100]
titles_100.shape

(100,)

In [13]:
# create gensin dictionary where every word has a unique numeric ID

### Word2Vec Model - CBOW

In [67]:
# source: https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py
import gensim.models

In [68]:
model = gensim.models.Word2Vec(sentences=tokenized_titles,
                               sg=0, # CBOW training algorithm
                               seed=1977,
                               iter=10
                              )

In [ ]:
# add new downloads to trained model

In [ ]:
# save trained model

In [ ]:
# similar word vectors


### Word2Vec Model - Skip-Grams